In [1]:
# 기본
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import seaborn as sns 
# import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold 
import warnings 
warnings.filterwarnings("ignore")

In [2]:
# module path 추가하는 함수 작성
def syspath(path):
  if path not in sys.path:
    sys.path.append(path)
  else : pass

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import warnings
import sys
# module path 추가
syspath('/content/drive/MyDrive/NA1/source_code/mod')
warnings.filterwarnings('ignore')
# EDA module에서 Analysis, Preprocess class 임포트
import EDA
from EDA import Analysis, Preprocess
from MLModeling import LightGBM

In [14]:
ndf=pd.read_pickle('/content/drive/MyDrive/NA1/source_code/demo/demo_0426.1/dataset/demo_dev_df_v0426.pkl')
ndf.head() # 라벨인코딩 전의 데이터셋

,R6M_CON_IMP_CNT,R6M_CON_IMP_AMT,R6M_CON_NUMB_OF_INSPECTION,R6M_CON_NUMB_OF_DETECTED,R6M_CON_RATIO_OF_DETECTED,R6M_CODE_IMP_CNT,R6M_CODE_IMP_AMT,R6M_CODE_NUMB_OF_INSPECTION,R6M_CODE_NUMB_OF_DETECTED,R6M_COUNTRY_IMP_CNT,R6M_COUNTRY_IMP_AMT,R6M_COUNTRY_NUMB_OF_INSPECTION,R6M_COUNTRY_NUMB_OF_DETECTED,R6M_COUNTRY_RATIO_OF_DETECTED,R6M_REP_IMP_CNT,R6M_REP_IMP_AMT,R6M_REP_NUMB_OF_INSPECTION,R6M_REP_NUMB_OF_DETECTED,R6M_REP_RATIO_OF_DETECTED,C_MONTH,A_HOUR,CON_ILLEGAL_RATIO,LABEL
0,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0,2,11,0.0,0
1,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0,2,11,0.0,0
2,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0,2,11,0.0,0
3,6,1395774.24,0,0,0.0,49,7322224.0,0,0,72,6.842362e+06,0,0,0.0,2695,1.915663e+09,0,0,0.0,2,15,0.0,0
4,6,1395774.24,0,0,0.0,49,7322224.0,0,0,72,6.842362e+06,0,0,0.0,2695,1.915663e+09,0,0,0.0,2,15,0.0,0


LABEL==1인 갯수 13959/ LABLE==0인 갯수 2132549

In [33]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
sss = StratifiedKFold(n_splits = 5, random_state = 156, shuffle = False) 
n_iter = 0

X = ndf.drop('LABEL', axis=1)
y = ndf.LABEL

cv_precision = []
cv_recall = []

for train_index, test_index in sss.split(X, y): # 학습-테스트 인덱스 추출
  print('[학습,테스트 인덱스 추출]')
  print("Train:", train_index, "Test:", test_index) 
  X_train, X_test = X.iloc[train_index], X.iloc[test_index] 
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  #trian, test 라벨 분포 (인덱스 번호로 5-fold 분할이 잘 되었는지 확인함)
  train_unique_label, train_counts_label = np.unique(y_train, return_counts = True) 
  test_unique_label, test_counts_label = np.unique(y_test, return_counts = True) 
  print() 
  print('Label Distributions: \n') 
  print(f' train: {np.round(train_counts_label/ len(y_train),4)}') # 0,1 비율
  print(f' test:  {np.round(test_counts_label/ len(y_test),4)}')

  # LGBMClassifier
  lgb_wrapper = LGBMClassifier(n_estimators=100, boost_from_average=False)
  evals = [(X_test, y_test)]
  lgb_wrapper.fit(X_train, y_train, early_stopping_rounds=20,
                  eval_set=evals, eval_metric='logloss', verbose='silence')
  preds = lgb_wrapper.predict(X_test)
  n_iter += 1

  confusion = confusion_matrix(y_test, preds)
  accuracy = accuracy_score(y_test, preds)
  precision = precision_score(y_test, preds)
  recall = recall_score(y_test, preds)
  f1 = f1_score(y_test, preds)
  print('오차 행렬')
  print(confusion)
  print('Accuracy : {0:.4f}, Precision : {1:.4f}, Recall : {2:.4f},\
            f1 score : {3:.4f}'.format(accuracy, precision, recall, f1)) 
  print('-' * 100) 
  print()
  cv_precision.append(precision)
  cv_recall.append(recall)
print('## 교차 검증별 precision:', np.round(cv_precision, 4))
print('## 교차 검증별 recall:', np.round(cv_recall, 4))

print('## 평균 검증 precision:',np.round(np.mean(cv_precision),4))
print('## 평균 검증 recall:',np.round(np.mean(cv_recall),4))

[학습,테스트 인덱스 추출]
Train: [ 427838  427839  427840 ... 2146405 2146406 2146407] Test: [     0      1      2 ... 896394 896395 896396]

Label Distributions: 

 train: [0.9935 0.0065]
 test:  [0.9935 0.0065]
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[75]	valid_0's binary_logloss: 0.0208796	valid_0's binary_logloss: 0.0208796
오차 행렬
[[426436     54]
 [  2080    712]]
Accuracy : 0.9950, Precision : 0.9295, Recall : 0.2550,            f1 score : 0.4002
----------------------------------------------------------------------------------------------------

[학습,테스트 인덱스 추출]
Train: [      0       1       2 ... 2146405 2146406 2146407] Test: [ 427838  427839  427840 ... 1113666 1113667 1113668]

Label Distributions: 

 train: [0.9935 0.0065]
 test:  [0.9935 0.0065]
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[53]	valid_0's binary_logloss: 0.025462	valid_0's binary_logloss: 0.025462
오차 행렬
[[426294

결론:
학습 데이터, 검증 데이터에서 0과 1의 비율을 동일하게 하기 위해 stratify 기능은 써야 한다. 하지만, 교차 검증을 할 시 precision과 recall이 매우 떨어지는 현상이 나타났다.
교차 검증은 1만개정도의 적은 데이터셋에 주로 사용하는 기법인데,
위와 같은 불균형 데이터셋에 교차 검증을 진행할 시, 성능 저하를 초래한다.